In [1]:
import os
import sys

import functions.io
import functions.graphs
import config

In [2]:
def get_sample_status(x):
    """Convenience function to create column with sample status"""

    status_int = int(x.split("_")[2])
    
    if status_int <= 3:
        status = 'CTRL'
    elif 3 < status_int <= 6:
        status = 'DIABETES'

    return status


def apply_color(x):
    """Convenience function to create a column of strings of colors"""

    if x is np.nan:
        col = 'black'
    else:
        col = 'red'

    return col

In [3]:
data = functions.io.read_all_samples(config.Config.DATA_DIRECTORY)

In [4]:
samples = functions.io.get_sample_names(data)

## Analyse Sample

In [ ]:
# choose the most frequent sequences and drop the rest
max_seqs = config.Config.MAX_SEQS
# join a pair of sequences only if they are within max_weight Levenshtein (edit) distance
max_weight = config.Config.MAX_WEIGHT

In [ ]:
for i in range(len(samples)):
    
    sample_data = data[data['sample']==samples[i]]
    seqs = sample_data['seq'].tolist()[0:max_seqs]
    counts = sample_data['count'].tolist()

    dists = functions.graphs.get_distances(seqs)
    coords = functions.graphs.get_dimension_mapping(dists)

    G = functions.graphs.get_graph(seqs, dists, max_weight=max_weight)
    functions.graphs.plot_graph(G, coords, seqs, counts, config.Config.OUTPUT_DIRECTORY+samples[i])
    
    node_degrees = sorted(G.degree, key=lambda x: x[1], reverse=True)